# Choosing an optimal distributed binary tree

As proven in [this](https://math.stackexchange.com/a/1462106) StackExchange response, there are $\frac{(y+x-1)!}{y!(x-1)!}$ possible combinations of x numbers from $\mathbb{N}_0$ that sum up to y.

If we define $n$ as the number of floating-point numbers that are to be summed up and $m$ as the largest rank index (cluster size minus 1), we therefore determine that the number of possible assignment of summands to ranks (including zero-assignments and permutations) is:
$$\frac{(n+m)!}{n! m!} = \frac{n! \cdot (n+1) \cdot ... \cdot (n+m)}{n!m!} = \frac{1}{m!} \Pi^m_{i=1} (n+i) $$

In [1]:
from math import factorial
from functools import reduce

n = 21410970 # number of per-site log likelihoods from TarvD7
m = 8 -1 # number of cores in my desktop CPU - 1


combinations = reduce(lambda x, y: x * y, [n + i for i in range(1, m + 1)]) / factorial(m)

combinations

4.092836842947645e+47

So, when calculating the sum of log-likelihoods of TarvD7 on my desktop PC, there are about $4 \cdot 10^{47}$ different 
binary trees we could use. That is a lot. If we still want to choose a decent tree, we need heuristics.

In [2]:
def parent(idx):
    return idx & (idx - 1)

def calcStartIndices(n_summands):
    idx = 0
    result = list()
    for n in n_summands:
        result.append(idx)
        idx += n
    return result

def rankFromIndex(idx, startIndices):
    guard = [(len(startIndices), 1)]
    offset = [(rank, startIdx - idx) for rank, startIdx in enumerate(startIndices)] + guard
    # find first positive valued offset
    offset_is_positive = lambda rank_offset_tuple: rank_offset_tuple[1] > 0
    rank = next(filter(offset_is_positive, offset))[0]
    return rank - 1


"""
Given a list of summands that are assigned to each rank, how many partial sums must be sent out to another rank.
"""
def rankIntersectingNumbers(n_summands, startIndices):
    n_parts = sum(n_summands)
    is_rank_intersecting = lambda idx: rankFromIndex(idx, startIndices) != rankFromIndex(parent(idx), startIndices)
    
    rank_intersecting_summands = filter(is_rank_intersecting, range(0, n_parts))
    return list(rank_intersecting_summands)

def rankIntersectionCount(n_summands, startIndices):
    is_rank_intersecting = lambda startIndex, idx: parent(idx) < startIndex
    # calculate a boolean list of rank intersecting numbers for each rank
    rankIntersectingFlagsPerRank = map(lambda t:
           [is_rank_intersecting(t[0], i) for i in range(t[0], t[0] + t[1])], zip(startIndices, n_summands))
    
    countTrueValues = lambda x: reduce(lambda acc, flag: acc + 1 if flag else acc, x, 0)
    return sum(map(countTrueValues, rankIntersectingFlagsPerRank))

In [3]:
n_summands = [3, 4, 23]
startIndices = calcStartIndices(n_summands)

rin = rankIntersectingNumbers(n_summands, startIndices)
print(rin)

[3, 4, 7, 8, 16]


Covering the whole search space is likely not feasible, since the calculation of rankIntersectingNumbers is non-trivial and the number of possible combinations really high.

In [4]:
from itertools import permutations

def possibleNSummands(clusterSize, numberOfParts):
    # Naive implementation after https://math.stackexchange.com/a/1462106
    t = ("*" * numberOfParts) + ("+" * clusterSize)
    for tuple_c in permutations(t, len(t)):
        c = "".join(tuple_c)
        n_summands = list(map(lambda x: len(x), c.split("+")))
        # relative percentage of summands
        legal = map(lambda x: (1.0 - ( x/ (numberOfParts / clusterSize)) <= 0.1), n_summands)
        if all(list(legal)):
            yield n_summands
        

In [5]:
%%script false --no-raise-error
# Even if left running 24/7, this piece of code probably wont finish in my lifetime (or the reader's)
# MORTALS, DO NOT RUN THIS CODE!
clusterSize = 2
number_of_parts = 214109
scores = [(rankIntersectionCount(n_summands, calcStartIndices(n_summands)), n_summands)
          for n_summands in possibleNSummands(clusterSize, number_of_parts)]

best_scores = sorted(scores, key=lambda t: t[0])
best = best_scores[0]
perfect = best[0] <= clusterSize

(perfect, best)

If we simply try to minimize the number of messages sent between ranks, the search algorithm will slyly place all numbers on rank 0, alleviating the communicator of sending any messages. In reality, this most likely will not be the fastest method, because the CPU is going to be the bottleneck instead of the network. Other performance metrics are needed to score different distributions.

## Modeling communication and computation

With [this naive benchmark](https://github.com/stelzch/allreduce-prep/blob/599fb8e75f7110b96bd1b59e503677b4481f2d89/benchmarks/mpi_send_bench.cpp), we can see that a send & receive can take as little as 281 ns on a shared-memory machine. Let us generously assume that this is the time used by sending a partial sum between two ranks. A simple addition of double values is shown by the following benchmark to take about 4.16ns, assuming all data does fit in the cache:

    Running ./benchmark
    Run on (8 X 4000 MHz CPU s)
    CPU Caches:
      L1 Data 16 KiB (x8)
      L1 Instruction 64 KiB (x4)
      L2 Unified 2048 KiB (x4)
      L3 Unified 8192 KiB (x1)
    Load Average: 1.82, 2.26, 2.41
    ***WARNING*** CPU scaling is enabled, the benchmark real time measurements may be noisy and will incur extra overhead.
    ----------------------------------------------------------------------
    Benchmark                            Time             CPU   Iterations
    ----------------------------------------------------------------------
    [...]
    BM_summation                      4.16 ns         4.15 ns    168624096
    
With these naive estimations, we build the following tree scoring function:

$$
Score = t_{\textrm{MPI_Send}} * n_{\textrm{rankIntersectingNumbers}} + max \{i \in [0, m]\, |\, n_i * t_{\textrm{double add}} \}
$$

where $n_{\textrm{rankIntersectingNumbers}}$ is the numbers of rank intersecting partial sums, $m$ the largest rank index and $n_i$ the number of summands assigned to rank $i$.

In [6]:
t_send = 281e-9
t_doubleadd = 4.15e-9

def score(x):
    return t_send * rankIntersectionCount(x, calcStartIndices(x)) + max(x) * t_doubleadd

n = 504850 # number of per-site log likelihoods from PeteD8

from IPython.display import HTML, display

def display_table(data, header = ["Name", "Assigned numbers per rank", "Score", "Number of messages"]):
    html = "<table>"
    if len(header) > 0:
        html += "<tr>" + "".join(["<th>" + h + "</th>" for h in header])
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

score_tuple = lambda desc, x: (desc, x, score(x), rankIntersectionCount(x, calcStartIndices(x)))
display_table([
    score_tuple("Evenly distributed", [n//4] * 4),
    score_tuple("All on rank 0", [n // 4 * 4, 0, 0, 0])
])

Name,Assigned numbers per rank,Score,Number of messages
Evenly distributed,"[126212, 126212, 126212, 126212]",0.0005313668,27
All on rank 0,"[504848, 0, 0, 0]",0.0020951192,0


Now we can see that it takes considerably more time when doing everything on rank 0, since we profit from our multiprocessing capabilities.

Let us use 256 cores now, and round the number of summands down to the greatest power of 2 that is still smaller than our fair share.

In [17]:
from math import log2, floor
floor_to_power_of_2 = lambda x: 2**floor(log2(x))

n = 504850 # number of per-site log likelihoods from PeteD8
m = 256 # Let's throw some cores at the problem

per_rank = floor_to_power_of_2(n // m)
power2_distribution = [per_rank] * m
power2_distribution[-1] += n - sum(power2_distribution) # remainder goes on last rank


even_distribution = [n//m] * m
for i in range(0, n % m):
    even_distribution[-i-1] += 1 #n - sum(even_distribution)

even_distribution_remainder_on_first = [n // m] * m
for i in range(0, n % m):
    even_distribution_remainder_on_first[i] += 1

    
assert(sum(power2_distribution) == sum(even_distribution) == sum(even_distribution_remainder_on_first))
reference=score_tuple("Evenly distributed", even_distribution)

display_table([
    reference,
    score_tuple("Evenly distributed (remainder on lower ranks)", even_distribution_remainder_on_first),
    score_tuple("Round down to power of 2", power2_distribution)
])

Name,Assigned numbers per rank,Score,Number of messages
Evenly distributed,"[1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973]",0.00040186895,1401
Evenly distributed (remainder on lower ranks),"[1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972]",0.00046902794999999995,1640
Round down to power of 2,"[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1

As we can see, the round down to power of 2 approach yielded the optimal amount of messages, but our model predicts a runtime twice as long as the even distribution, probably because of the imbalance in the distribution of the computational work load.

Next, lets try to reduce the number of message by clearing least-significant zeros of the starting indices until we reach a certain threshold.

In [18]:
# fair share is the number of summands we would have been assigned if they were evenly distributed.
# We accept differences less than or equal to 5%
fairShare = n / m
deviationWithinBounds = lambda share, dev, n, m: (1.0 - dev) <= share / (n / m) <= (1.0 + dev)
def optimizeIdx(prevIdx, currentIdx, n, m, dev = 0.05):
    proposedIdx = currentIdx
    while prevIdx < proposedIdx and deviationWithinBounds(proposedIdx - prevIdx, dev, n, m):
        currentIdx = proposedIdx # if our proposal is accepted, set the current index to the proposed index
        proposedIdx = parent(currentIdx) # optimize our index by setting it to its parent
    return currentIdx

startIndices = [0]
for rank in range(1, m):
    last = startIndices[-1]
    startIndices.append(optimizeIdx(last, last + int(fairShare), n, m))

# Get list of summands per rank and add remaining summands to last rank
def startIndices2nSummands(n, x):
    extended = x + [n]
    return map(lambda t: t[1] - t[0], zip(x, extended[1:]))

lsbclear_distribution = list(startIndices2nSummands(n, startIndices))
display_table([
    reference,
    score_tuple("LSB clearing", lsbclear_distribution)
])

Name,Assigned numbers per rank,Score,Number of messages
Evenly distributed,"[1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973]",0.00040186895,1401
LSB clearing,"[1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 15250]",0.0002745995,752


Nice, while the number of messages is not optimal, the runtime score is now significantly lower and even beats the even distribution.

Of course, this not necessarily indicative of any real-world work loads.
Let us test with the same number of cores (256) on a much smaller dataset (NagyA1.psllh with 171998 sites):

In [20]:
n = 171998
m = 256
fairShare = n / m

even_distribution = [n//m] * m
even_distribution[0] += n - sum(even_distribution)

startIndices = [0]
for rank in range(1, m):
    last = startIndices[-1]
    startIndices.append(optimizeIdx(last, last + int(fairShare), n, m))
    
lsbclear_distribution = list(startIndices2nSummands(n, startIndices))
display_table([
    score_tuple("Even distribution", even_distribution),
    score_tuple("LSB clearing", lsbclear_distribution[::-1])
])

Name,Assigned numbers per rank,Score,Number of messages
Even distribution,"[893, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671]",0.00040946995,1444
LSB clearing,"[8798, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640]",0.0003324047,1053


Again, in this case, our LSB clearing beats the even distribution! In this model, it is about 1.23x faster.

Another optimization we could do is not to put the remainder all on one node, but instead spread it across all nodes:

In [21]:
even_distribution_remainder_spread = [n//m] * m
for i in range(n % m):
    even_distribution_remainder_spread[i] += 1
assert(sum(even_distribution_remainder_spread) == n)

display_table([
    score_tuple("Even distribution", even_distribution),
    score_tuple("Even distribution with spread remainder", even_distribution_remainder_spread)
])

Name,Assigned numbers per rank,Score,Number of messages
Even distribution,"[893, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671]",0.00040946995,1444
Even distribution with spread remainder,"[672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671, 671]",0.00025259779999999997,889


For our smaller example (NagyA1), this did not improve the runtime, but what about TarvD1?

In [24]:
n = 504850
p = 256

even_distribution = [n//m] * m
even_distribution[0] += n - sum(even_distribution)

even_distribution_remainder_spread = [n//m] * m
for i in range(n % m):
    even_distribution_remainder_spread[i] += 1

display_table([
    score_tuple("Even distribution", even_distribution),
    score_tuple("Even distribution with spread remainder", even_distribution_remainder_spread)
])

Name,Assigned numbers per rank,Score,Number of messages
Even distribution,"[1990, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972]",0.00046881749999999996,1639
Even distribution with spread remainder,"[1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972]",0.00046902794999999995,1640


Now, let us combine our two approaches and devise an even distribution with a spread remainder that is optimized to increase the number of least-significant zeros:

In [31]:
# optimized even
opt_even = calcStartIndices(even_distribution_remainder_spread.copy())
prev_idx = opt_even[0]
for i, idx in enumerate(opt_even):
    if i == 0:
        continue
    opt_even[i] = optimizeIdx(prev_idx, idx, n, m, dev=0.2)
    prev_idx = opt_even[i]

opt_even = list(startIndices2nSummands(n, opt_even))
assert(sum(opt_even) == sum(even_distribution_remainder_spread))

display_table([
    score_tuple("Even distribution with spread remainder and optimized Index", opt_even),
    score_tuple("Even distribution with spread remainder", even_distribution_remainder_spread)
]) 

Name,Assigned numbers per rank,Score,Number of messages
Even distribution with spread remainder and optimized Index,"[1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2397, 1699, 2048, 2048, 2048, 1792, 1792, 2384, 1712, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2402, 1694, 2048, 2048, 2048, 1792, 1792, 2382, 1714, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2398, 1698, 2048, 2048, 2048, 1792, 1792, 2378, 1718, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2394, 1702, 2048, 2048, 2048, 1792, 1792, 2374, 1722, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2390, 1706, 2048, 2048, 2048, 1792, 1792, 2370, 1726, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2406, 1690, 2048, 2048, 2048, 1792, 1792, 2386, 1710, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2402, 1694, 2048, 2048, 2048, 1792, 1792, 2382, 1714, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2398, 1698, 2048, 2048, 2048, 1792, 1792, 2378, 1718, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2394, 1702, 2048, 2048, 2048, 1792, 1792, 2374, 1722, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 2048, 1792, 1792, 2048, 2048, 2048, 2048, 1792, 1792, 2390, 1706, 2048, 2048, 2048, 1792, 1792, 2370, 1726, 2048, 2048, 2066]",0.0001844859,621
Even distribution with spread remainder,"[1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1973, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972]",0.00046902794999999995,1640


Nice, that is **a 1.55x speedup** compared to the even distribution, while using **less than a third of the messages**.